### Importação das bibliotecas

In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2021-12-29 14:36:40.989491: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


### Configurando os caminhos (paths)

In [2]:
dataset_path_new = "./cats_and_dogs_filtered"

In [3]:
train_path = os.path.join(dataset_path_new, "train")
validation_path = os.path.join(dataset_path_new, "validation")

## Definindo os modelos

In [4]:
img_shape = (150, 150, 3)

### MobileNetV2 model

In [5]:
mobileNet = tf.keras.applications.MobileNetV2(input_shape = img_shape, include_top = False, weights = "imagenet")

mobileNet.summary()

2021-12-29 14:36:42.060744: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-12-29 14:36:42.061337: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-12-29 14:36:42.088817: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-29 14:36:42.088970: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: NVIDIA GeForce GTX 1060 6GB computeCapability: 6.1
coreClock: 1.8095GHz coreCount: 10 deviceMemorySize: 5.93GiB deviceMemoryBandwidth: 178.99GiB/s
2021-12-29 14:36:42.088987: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-12-29 14:36:42.090280: I tensorflow/stream_executor/platfo

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 75, 75, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 75, 75, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 75, 75, 32)   0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [6]:
inceptionV3 = tf.keras.applications.InceptionV3(input_shape=img_shape, weights='imagenet', include_top=False)

inceptionV3.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [7]:
xception = tf.keras.applications.Xception(input_shape=img_shape, include_top=False, weights='imagenet')

xception.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 74, 74, 32)   864         input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 74, 74, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 74, 74, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [8]:
# agrupando os modelos em uma lista
models = [mobileNet, inceptionV3, xception]

### Congelando o modelo base e definindo o cabeçalho personalizado, utilizando para a transferência de aprendizagem

In [9]:
def cabecalho_transfer_learning(model):
    """Retorna o cabeçalho personalizado da rede pré treinada"""
    model.trainable = False

    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(model.output)

    prediction_layer = tf.keras.layers.Dense(units = 512, activation = 'relu')(global_average_layer) 
    prediction_layer = tf.keras.layers.Dense(units=256, activation='relu')(prediction_layer)
    prediction_layer = tf.keras.layers.Dense(units=128, activation='relu')(prediction_layer)
    prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(prediction_layer) #classificação binária

    return prediction_layer

### Criando e definindo o modelo

In [10]:
models_final = []
for model in models:
    models_final.append(tf.keras.models.Model(inputs = model.input, outputs = cabecalho_transfer_learning(model=model)))

### Compilando o modelo

usando as mesma configs para compilar os modelos

In [11]:
for model in models_final:
    model.compile(optimizer=tf.keras.optimizers.RMSprop(lr = 0.0001), loss="binary_crossentropy", metrics = ["accuracy"])

### Criando o dataset com o generator

aplicando a normalização nas imagens

In [12]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [13]:
train_generator = data_gen_train.flow_from_directory(train_path, target_size=(128,128), batch_size=128, class_mode="binary")
valid_generator = data_gen_train.flow_from_directory(validation_path, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


### Treinando o modelo

In [14]:
for idx, model in enumerate(models_final):
    print('Treinando o modelo: ', models[idx].name)
    model.fit_generator(train_generator, epochs=8, validation_data=valid_generator)

Treinando o modelo:  mobilenetv2_1.00_224


/home/lps/anaconda3/envs/ia/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021-12-29 14:36:45.877839: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-12-29 14:36:45.894491: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3699740000 Hz


Epoch 1/8


2021-12-29 14:36:47.411150: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-12-29 14:36:47.670910: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2021-12-29 14:36:48.050043: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2021-12-29 14:36:48.101101: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


16/16 [==============================] - 31s 1s/step - loss: 0.4015 - accuracy: 0.8073 - val_loss: 0.1776 - val_accuracy: 0.9160
Epoch 2/8
16/16 [==============================] - 6s 350ms/step - loss: 0.1590 - accuracy: 0.9311 - val_loss: 0.1723 - val_accuracy: 0.9250
Epoch 3/8
16/16 [==============================] - 6s 357ms/step - loss: 0.1096 - accuracy: 0.9603 - val_loss: 0.1568 - val_accuracy: 0.9330
Epoch 4/8
16/16 [==============================] - 6s 357ms/step - loss: 0.0857 - accuracy: 0.9684 - val_loss: 0.1498 - val_accuracy: 0.9350
Epoch 5/8
16/16 [==============================] - 5s 334ms/step - loss: 0.0710 - accuracy: 0.9795 - val_loss: 0.1625 - val_accuracy: 0.9310
Epoch 6/8
16/16 [==============================] - 5s 341ms/step - loss: 0.0485 - accuracy: 0.9921 - val_loss: 0.1547 - val_accuracy: 0.9330
Epoch 7/8
16/16 [==============================] - 5s 332ms/step - loss: 0.0382 - accuracy: 0.9932 - val_loss: 0.1478 - val_accuracy: 0.9380
Epoch 8/8
16/16 [========

### Avaliação dos modelos com transferência de aprendizagem

In [15]:
validacao_transferencia = {}
for idx, model in enumerate(models_final):
    validacao_transferencia[models[idx].name] = model.evaluate_generator(valid_generator)[1] #pegando somente a acurácia

validacao_transferencia

/home/lps/anaconda3/envs/ia/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


{'mobilenetv2_1.00_224': 0.9350000023841858,
 'inception_v3': 0.8939999938011169,
 'xception': 0.9440000057220459}

## Fine tuning
- Use Fine Tuning em apenas uma parte d rede. A ideia é pegar uma parte específica da rede neural para o problema específico;
- Faça o processo de fine tuning após a transferência de aprendizagem. Caso tente antes, os gradientes serão muito diferentes entre o cabeçalho personalizado e algumas camadas descongeladas do modelo base

### Descongelando algumas camadas para fazer o treinamento de acordo com o problema

In [16]:
for model in models:
    #descongelando as camadas
    model.trainable = True

    print('total camadas do modelo {}: {}'.format(model.name, len(model.layers)))

total camadas do modelo mobilenetv2_1.00_224: 154
total camadas do modelo inception_v3: 311
total camadas do modelo xception: 132


### Escolhendo a partir de qual camada da rede começar a fazer o treinamento

Utilizando as ultimas 30 camadas para o treinamento

In [17]:
for model in models:
    for layer in model.layers[:len(model.layers) - 30]:
        layer.trainable = False

### Compilando os modelos para o fine tuning

In [18]:
for model in models_final:
    model.compile(optimizer=tf.keras.optimizers.RMSprop(lr = 0.0001), loss="binary_crossentropy", metrics=["accuracy"])

### Fine tuning

In [19]:
for idx, model in enumerate(models_final):
    print('Treinando o modelo: ', models[idx].name)
    model.fit_generator(train_generator, epochs=8, validation_data=valid_generator)

Treinando o modelo:  mobilenetv2_1.00_224
Epoch 1/8
16/16 [==============================] - 13s 519ms/step - loss: 0.3351 - accuracy: 0.8426 - val_loss: 0.3148 - val_accuracy: 0.9210
Epoch 2/8
16/16 [==============================] - 5s 328ms/step - loss: 0.0476 - accuracy: 0.9917 - val_loss: 0.2512 - val_accuracy: 0.9410
Epoch 3/8
16/16 [==============================] - 5s 321ms/step - loss: 0.0320 - accuracy: 0.9951 - val_loss: 0.3044 - val_accuracy: 0.9350
Epoch 4/8
16/16 [==============================] - 5s 323ms/step - loss: 0.0056 - accuracy: 1.0000 - val_loss: 0.3421 - val_accuracy: 0.9330
Epoch 5/8
16/16 [==============================] - 5s 328ms/step - loss: 0.0088 - accuracy: 0.9956 - val_loss: 0.4558 - val_accuracy: 0.9350
Epoch 6/8
16/16 [==============================] - 5s 332ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.4888 - val_accuracy: 0.9360
Epoch 7/8
16/16 [==============================] - 5s 323ms/step - loss: 7.6878e-04 - accuracy: 1.0000 - val_lo

### Avaliação do modelo com fine tuning

In [20]:
validacao_fine_tuning = {}

for idx, model in enumerate(models_final):
    validacao_fine_tuning[models[idx].name] = model.evaluate_generator(valid_generator)[1] # pegando somente a acurácia

validacao_fine_tuning

{'mobilenetv2_1.00_224': 0.9369999766349792,
 'inception_v3': 0.9200000166893005,
 'xception': 0.953000009059906}